In [1]:
from google.colab import drive
drive.mount("/content/MyDrive/")

Mounted at /content/MyDrive/


In [2]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 60kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=459cfb5b9266cabfef94101c584d9305f1aa2ff471ba90458f189281d406f8f7
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [3]:
dir_path = "/content/MyDrive/MyDrive/PFE_Deepfakes/datasets/DFDC/" 
train_path = "datasets/DFDC/temporal/train"
validation_path = "datasets/DFDC/temporal/validation"
test_path = "datasets/DFDC/temporal/test"

In [4]:
cd /content/MyDrive/MyDrive/PFE_Deepfakes

/content/MyDrive/MyDrive/PFE_Deepfakes


In [5]:
import os
import json
import numpy as np
import time
import face_recognition
from df_code.CViT_main.CViT_main.helpers import helpers_face_extract_1, helpers_read_video_1, blazeface
import cv2
import shutil
from time import perf_counter
import sys
import torch
from df_code.CViT_main.CViT_main.helpers.blazeface import BlazeFace

In [ ]:
#data
metadata = json.load(open('datasets/DFDC/dfdc_train_part_00/dfdc_train_part_0/metadata.json'))
fake=[]
original=[]
    
for file_dp in metadata :
  print( metadata[file_dp])
  print(('original' in metadata[file_dp]) and (metadata[file_dp]['original'] not in original) and (metadata[file_dp]['original'] is not None))
  if (('original' in metadata[file_dp]) and (metadata[file_dp]['original'] not in original) and (metadata[file_dp]['original'] is not None)):
            original.append(metadata[file_dp]['original'])
            fake.append(file_dp)
print(fake)
print(original)
print(np.array([[i, j] for i, j in zip(fake, original)]).ravel())

In [ ]:


# code used to extract DFDC dataset


  
# load DFDC json
def load_metadata(dir_path):
    metafile = dir_path+'/metadata.json'
     
    if os.path.isfile(metafile):
        with open(metafile) as data_file:
            data = json.load(data_file)
    else:
        return 1
    
    return data

def extract_face(dir_path):
    print("beginning...")
    for item in sorted(os.listdir(dir_path)): # iterate over the 49 train part of DFDC
        print(item)
        
        file_num = int(item[16:]) #by default the destination is the train_path
        destination = train_path
        
        if (file_num > 34 and file_num <46): #but part 35 to 45 will go to the validation set
            destination = validation_path 
        
        if (file_num > 45): #and part 46 to 49 to test set
            destination = test_path
        
        meta_full_path = os.path.join(dir_path, item) #meta_full_path is the path to each of the 49 part of the original dataset
        
        if os.path.isdir(meta_full_path) <= 3 : #there could be other files in our dir_path that are not our folders with the train data
            data = load_metadata(meta_full_path+'/dfdc_train_part_'+str(file_num))
            print(meta_full_path+'/dfdc_train_part_'+str(file_num))
            
            if data != 1: #if the metadata files exists (look the returns of the above function)
                
                if not os.path.exists(destination+str(file_num)): #check if exists else creates directory
                    os.makedirs(destination+str(file_num))
                
                if not os.path.isfile(destination+str(file_num)+'/metadata.json'): #check if exists else creates metadata
                    shutil.copy2(dir_path+'/'+item+'/dfdc_train_part_'+str(file_num)+'/metadata.json', destination+str(file_num)+'/metadata.json')
                  
                filtered_files = filter_unique_files(data)
                
                for filename in filtered_files:
                    # check if the file name is found in metadata, and its label
                    if filename.endswith(".mp4") and os.path.isfile(dir_path+'/'+item+'/dfdc_train_part_'+str(file_num)+'/'+filename):
                        label = data[filename]['label'].lower()
                        # append fake video names with their corresponding real video names
                        original = ''
                        if data[filename]['label'].lower() == 'fake':
                            original = '_'+data[filename]['original'][:-4]
                        image_path = destination+str(file_num)+'/'+label
                        if not os.path.exists(image_path):
                            os.makedirs(image_path)
                            
                        process_video(dir_path+'/'+item+'/dfdc_train_part_'+str(file_num)+'/'+filename, filename, image_path, original)

# access video
def process_video(video_path, filename, image_path, original):
    gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    facedet = BlazeFace().to(gpu)
    facedet.load_weights("df_code/CViT_main/CViT_main/helpers/blazeface.pth")
    facedet.load_anchors("df_code/CViT_main/CViT_main/helpers/anchors.npy")
    _ = facedet.train(False)
    
    from df_code.CViT_main.CViT_main.helpers.helpers_read_video_1 import VideoReader
    from df_code.CViT_main.CViT_main.helpers.helpers_face_extract_1 import FaceExtractor

    frames_per_video = 10
     
    video_reader = VideoReader()
    video_read_fn = lambda x: video_reader.read_random_frames(x, num_frames=frames_per_video)
    face_extractor = FaceExtractor(video_read_fn, facedet)
    
    faces = face_extractor.process_video(video_path)
    # Only look at one face per frame.
    face_extractor.keep_only_best_face(faces)
    n = 0
    for frame_data in faces:
        for face in frame_data["faces"]:
            face_locations = face_recognition.face_locations(face)
            for face_location in face_locations:
            
                top, right, bottom, left = face_location
                face_image = face[top:bottom, left:right]
                resized_face = cv2.resize(face_image, (224, 224), interpolation=cv2.INTER_AREA)
                resized_face = cv2.cvtColor(resized_face, cv2.COLOR_RGB2BGR)
            
                cv2.imwrite(image_path+"/"+filename[:-4]+original+"_"+str(n)+".jpg",resized_face, [int(cv2.IMWRITE_JPEG_QUALITY), 85])
                
                n += 1

def filter_unique_files(metadata):
    fake=[]
    original=[]
    
    for file_dp in metadata:
        if (('original' in metadata[file_dp]) and (metadata[file_dp]['original'] not in original) and (metadata[file_dp]['original'] is not None)): #allows to take only one fake per original video (real)
            original.append(metadata[file_dp]['original'])
            fake.append(file_dp)
    return np.array([[i, j] for i, j in zip(fake, original)]).ravel()

start_time = perf_counter()
extract_face(dir_path)
end_time = perf_counter()
print("--- %s seconds ---" % (end_time - start_time))